# Hybrid search with DocArray + Weaviate

In this notebook, we will set up a Weaviate document index with 200 movie datapoints, in order to do some fun searching with on this store with DocArray! We will cover 3 different searches:
  1. Pure text search (symbolic search with BM25)
  2. Vector search and
  3. Hybrid search (combining symbolic and vector approaches)
  
Make sure you have a docker instance with Weaviate running locally, or set credentials for connecting to a remote instance. 

In [1]:
import docarray
assert docarray.__version__ == '0.30.0'  # verify that we have 0.30.0 here :)

In [2]:
import os
import numpy as np
import csv
import requests as rq
from typing import Dict
from pydantic import Field
from docarray import BaseDoc
from docarray.typing import NdArray
from docarray.index.backends.weaviate import WeaviateDocumentIndex

In [3]:
import warnings
warnings.filterwarnings('ignore')

## Data loading and encoding

We will be using the IMDB Movies Dataset from Kaggle, containing the 1000 top rated movies on IMDB. To download this dataset, we will use `opendatasets`, which will download the data as a csv file into the following directory structure: 

`imdb-dataset-of-top-1000-movies-and-tv-shows/imdb_top_1000.csv`

### Document Schema
After the data is downloaded, we define a DocArray document schema for our data, consisting of a text field (concatenation of title, overview and actors names of the movies) and an embedding field. 

In [4]:
# Define a docarray document schema
class MovieDocument(BaseDoc):
    text: str  # will contain a concatenation of title, overview and actors names
    embedding: NdArray[384] = Field(is_embedding=True)  # Embedding column -> vector representation of the document


### Encoding
We also define a helper function `encode()` to help us encode our text fields using `sentence-transformers`. We will be using the `'multi-qa-MiniLM-L6-cos-v1'` model, which has been specifically trained for semantic search, using 215M question-answer pairs from various sources and domains, including StackExchange, Yahoo Answers, Google & Bing search queries and many more. This model should perform well across many search tasks and domains, including the movies domain. Let's see how well it does.

In [5]:
import opendatasets as od
od.download("https://www.kaggle.com/datasets/harshitshankhdhar/imdb-dataset-of-top-1000-movies-and-tv-shows")

Skipping, found downloaded files in "./imdb-dataset-of-top-1000-movies-and-tv-shows" (use force=True to force download)


In [6]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1') # may take some time to load!

def encode(text: str):
    embedding = model.encode(text)
    return embedding

In [7]:
# Load the data
docs = []
with open('imdb-dataset-of-top-1000-movies-and-tv-shows/imdb_top_1000.csv', newline='') as csvfile:
    movie_reader = csv.reader(csvfile, delimiter=',')
    h = movie_reader.__next__()
    c = 0
    for row in movie_reader:
        text = ' '.join([row[1], row[2], row[5], row[7]] + row[9:13])
        embedding = encode(text=text)
        d = MovieDocument(text=text, embedding=embedding, id=c)
        docs.append(d)
        c += 1

In [8]:
# Take a look at the loaded + processed documents
print("Number of documents: ", len(docs))
docs[0].summary()

Number of documents:  1000


📄 MovieDocument : 0 ...
╭────────────────────┬────────────────────────────────────────────────────────────────────────────────────────────╮
│ Attribute          │ Value                                                                                      │
├────────────────────┼────────────────────────────────────────────────────────────────────────────────────────────┤
│ text: str          │ The Shawshank Redemption 1994 Drama Two imprisoned ... (length: 207)                       │
│ embedding: NdArray │ NdArray of shape (384,), dtype: float32                                                    │
╰────────────────────┴────────────────────────────────────────────────────────────────────────────────────────────╯

## Indexing

Next, we want to store our documents in a document store, in this case in a Weaviate document index. This notebook assumes you have a local Weaviate instance set up, and therefore uses the host `http://localhost:8080`, but if you have instead set up a remote instance, please follow the [documentation](https://docs.docarray.org/user_guide/storing/index_weaviate/) for connecting to your Weaviate index.

In [9]:
batch_config = {
    "batch_size": 20,
    "dynamic": False,
    "timeout_retries": 3,
    "num_workers": 1,
}

runtimeconfig = WeaviateDocumentIndex.RuntimeConfig(batch_config=batch_config)

store = WeaviateDocumentIndex[MovieDocument](host="http://localhost:8080", index_name="MovieDocument")

store.configure(runtimeconfig)  # Batch settings being passed on
store.index(docs)

__________________

## 1. Text Search

Now, we can start to search our IMDB Movies data! First, let's use symbolic search with a text query to find some results. The `text_search()` method on the Weaviate document store will find exact matches (case insensitive) for our text query, meaning the words must appear in the documents text in order to be returned as a result.

In [10]:
# Define text query
text_query = 'monster scary'

In [11]:
text_q = store.build_query().text_search(text_query, search_field="text").limit(10).build()
text_results = store.execute_query(text_q)
print("Number of documents returned: ", len(text_results))

Number of documents returned:  3


In [12]:
for doc in text_results:
    print("Document number: ", doc.id)
    print("Document text:")
    print(doc.text)
    print("Match found at position ", doc.text.lower().find("monster"), "/", doc.text.lower().find("scary"))
    print()

Document number:  719
Document text:
Frankenstein 1931 Drama, Horror, Sci-Fi Dr. Frankenstein dares to tamper with life and death by creating a human monster out of lifeless body parts. James Whale Colin Clive Mae Clarke Boris Karloff
Match found at position  113 / -1

Document number:  401
Document text:
Beauty and the Beast 1991 Animation, Family, Fantasy A prince cursed to spend his days as a hideous monster sets out to regain his humanity by earning a young woman's love. Gary Trousdale Kirk Wise Paige O'Hara Robby Benson
Match found at position  100 / -1

Document number:  716
Document text:
Bride of Frankenstein 1935 Drama, Horror, Sci-Fi Mary Shelley reveals the main characters of her novel survived: Dr. Frankenstein, goaded by an even madder scientist, builds his monster a mate. James Whale Boris Karloff Elsa Lanchester Colin Clive
Match found at position  178 / -1



### Notes
Because only exact matches are returned, we only get 3 results, even though we set the limit to 10. This means, the text search does not automatically return "similar" results - nope! It only returns results that contain our term(s). Unfortunately, there are no exact matches for the term "scary" in our dataset.

__________________

## 2. Vector Search
Next, we'll try to use the embeddings we got from the inference endpoint, by also encoding the query and then computing similarity between our query vector and the vectors of our documents. We call the `find()` method with a query embedding on our document store.

In [13]:
q_embedding = encode(text_query)
vector_q = (
    store.build_query()
    .find(q_embedding)
    .limit(10)
    .build()
)

vector_results = store.execute_query(vector_q)
print("Number of documents returned: ", len(vector_results))

Number of documents returned:  10


### Notes 

Great! We have 10 results now. That's because the vector comparison can still find "similar" results even if there are no explicit mentions of our terms "monster" and "scary". We'll take a closer look at the results in a comparative analysis below.

__________________

## 3. Hybrid Search

In [15]:
q_embedding = encode(text_query)
hybrid_q = (
    store.build_query()
    .text_search(
        text_query, search_field=None  # Set as None as it is required but has no effect
    )
    .find(q_embedding)
    .limit(10)
    .build()
)

hybrid_results = store.execute_query(hybrid_q)
print("Number of documents returned: ", len(hybrid_results))

Number of documents returned:  10


### Notes

We again have a total of 10 results. These results should combine the best of both worlds, pushing documents to the top that contain an explicit mention of our search terms, and filling up the rest of the results with highly relevant results, possibly containing synonyms and related topics. We'll analyse this below!

__________________

## Results: A Comparative Analysis

Now that we have a bunch of search results from our various approaches, let's see how many relevant results were retrieved using the approaches.

In [16]:
from IPython.display import HTML
from typing import List
import pandas as pd
pd.set_option('display.max_colwidth', 0)

In [17]:
def display_result(bm25_data: List[MovieDocument], vec_data: List[MovieDocument], hyb_data: List[MovieDocument]):
    df1 = bm25_data.to_dataframe()
    df2 = vec_data.to_dataframe()
    df3 = hyb_data.to_dataframe()
    df = pd.concat([df1, df2, df3], axis=1).drop(["embedding"], axis=1)
    df.columns = [
        ["text search results", "text search results", "vector search results", "vector search results", "hybrid search results", "hybrid search results"],
        ["text", "id", "text", "id", "text", "id"]
    ]
    return df.style.set_table_styles(
    [{"selector": "", "props": [("border", "1px solid grey")]},
      {"selector": "tbody td", "props": [("border", "1px solid grey")]},
     {"selector": "th", "props": [("border", "1px solid grey")]}
    ]
)

In [18]:
display_result(text_results, vector_results, hybrid_results)

As we already discussed above, the **text search** using BM25 only returns results containing and exact match for our query terms, in this case only matching three documents. We know with certainty that these three documents are relevant to our query, making the *precision* of this approach very high. However, many documents are missing, even ones that contain a pluralized form of our search terms (eg. "monsters"), making the *recall* of this approach poor.

The top 5 **vector search** results are also all relevant to our query, containing mentions of "monsters", "beasts", "Frankenstein" (also a 'monster') a "shape-shifting alien" and "ogre", though only two of the three documents retrieved by the text search approach appear in the 10 results retrieved using vector search (documents 719 and 401). "Bride of Frankenstein" seems to be missing in the top 10. There are also some documents returned that don't seem to be relevant, for example documents 58, 740 and 984 (though they may still be 'scary'? Actually, no. "The Muppet Movie" and "Wreck-It Ralph" are certainly not scary). Nonetheless, the vector search approach here does quite well as it goes on latent representations of our data, capturing content with similar meanings such as `beasts, threat, ogre, victims` and is thereby able to successfully retrieve quite a few relevant documents (7 out of 10), with only a few non-relevant entries. 

Let's finally take a look at the **hybrid search** results, the top 5 of which contain our exact matches (documents 719, 401, 716) from the text search approach! This is because these documents got a higher combined score from the BM25 component in our hybrid scoring calculation. The rest of the results are filled in by the vector search component, showing improved results over a simple BM25 search, but still incorporating the advantages of this symbolic search approach into the hybrid search. Although "The Muppet Movie" and "Wreck-It Ralph" (our non-relevant results) still appear in the top 10, they've been pushed down the list, making way for more relevant search results.